### Date Preproscessing

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa
import librosa.display
import IPython.display as ipd

In [3]:
metadata = pd.read_csv('musicnet_metadata.csv')
train_data_files = glob('musicnet/musicnet/train_data/*.wav')
test_data_files = glob('musicnet/musicnet/test_data/*.wav')

In [4]:
def wav_to_mel_spec(path):
  y, sr = librosa.load(path)
  spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=512)
  return np.abs(librosa.amplitude_to_db(spec, ref=np.max))

In [5]:
train_data = [wav_to_mel_spec(path) for path in train_data_files]
test_data = [wav_to_mel_spec(path) for path in test_data_files]

In [6]:
train_data_ids = [int(path[-8:-4]) for path in train_data_files]
test_data_ids = [int(path[-8:-4]) for path in test_data_files]

train_labels = [metadata[metadata['id'] == i]['ensemble'].values[0] for i in train_data_ids]
test_labels = [metadata[metadata['id'] == i]['ensemble'].values[0] for i in test_data_ids]

In [7]:
labels_to_nums = {label: i for i, label in enumerate(set(train_labels))}
nums_to_labels = {i: label for label, i in labels_to_nums.items()}

### Model

In [8]:
import tensorflow as tf

In [12]:
x_train = np.array([x[:, :1024].reshape(512, 1024, 1) for x in train_data], np.float32)
x_test = np.array([x[:, :1024].reshape(512, 1024, 1) for x in test_data], np.float32)
y_train = np.array([labels_to_nums[label] for label in train_labels], np.float32)
y_test = np.array([labels_to_nums[label] for label in test_labels], np.float32)

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 1024, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(16, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(21, activation='relu'),
])

optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.001)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['acc'])

In [14]:
model.fit(x_train, y_train, epochs=20)

Epoch 1/10
10/10 [==============================] - 18s 2s/step - loss: 21.2085 - acc: 0.2406
Epoch 2/10
10/10 [==============================] - 17s 2s/step - loss: 2.9022 - acc: 0.4781
Epoch 3/10
10/10 [==============================] - 17s 2s/step - loss: 2.5816 - acc: 0.4781
Epoch 4/10
10/10 [==============================] - 17s 2s/step - loss: 2.4141 - acc: 0.4781
Epoch 5/10
10/10 [==============================] - 17s 2s/step - loss: 2.1040 - acc: 0.5125
Epoch 6/10
10/10 [==============================] - 18s 2s/step - loss: 1.8378 - acc: 0.6156
Epoch 7/10
10/10 [==============================] - 17s 2s/step - loss: 1.5002 - acc: 0.6594
Epoch 8/10
10/10 [==============================] - 17s 2s/step - loss: 1.2446 - acc: 0.6750
Epoch 9/10
10/10 [==============================] - 17s 2s/step - loss: 1.0797 - acc: 0.7156
Epoch 10/10
10/10 [==============================] - 17s 2s/step - loss: 1.0074 - acc: 0.7156


In [15]:
model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 192ms/step - loss: 1.9854 - acc: 0.4000


[1.9854122400283813, 0.4000000059604645]